In [ ]:
from __future__ import print_function, division
import os
import copy
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms, utils

### Seting up the dataloader

Implementing (Inheriting) the dataset class

In [ ]:
class CactusImageDataset(Dataset):
    "Aerial Cactus Classification Dataset"
    
    def __init__(self, csv_file, root_dir, transform = None):
        """
        Args:
            csv_file (string): Path to csv file with annotations
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.cactus_annotations = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
    
    def __len__(self):
        return self.cactus_annotations.shape[0]
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        img_name = os.path.join(self.root_dir,
                                self.cactus_annotations.iloc[idx, 0])
        image = io.imread(img_name)
        has_cactus = self.cactus_annotations.iloc[idx, 1]
        
        if self.transform:
            image = self.transform(image)
            
        sample = {"image":image, 'label':has_cactus}
        return sample
    
        

In [ ]:
image_transforms = {
    'train': transforms.Compose([
        transforms.ToPILImage(),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.RandomRotation(10),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
    ]),
    'test':transforms.Compose([
        transforms.ToPILImage(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
    ])
}

In [ ]:
dataset = CactusImageDataset('data/train.csv','data/train/', image_transforms['train'])

In [ ]:
train_size = int((0.80 * dataset.__len__()))
dev_size = dataset.__len__() - train_size
train_set, dev_set = random_split(dataset, [train_size, dev_size])

In [ ]:
BATCH_SIZE = 32
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
dev_loader = DataLoader(dev_set, batch_size=BATCH_SIZE, shuffle=True)

### Defining the Model

In [ ]:
class CactusIdentifierBase(nn.Module):
    """
        Aerial Cactus Identifier Model
    """
    def __init__(self):
        super(CactusIdentifier, self).__init__()
        self.conv1 = nn.Conv2d(3,8,3,1,1)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(8,32,3,1,1)
        self.fc1 = nn.Linear(8*8*32, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64,1)
        self.act = nn.ReLU()
    
    def forward(self, inp_image):
        out = self.pool(self.act(self.conv1(inp_image)))
#         print(out.shape)
        out = self.pool(self.act(self.conv2(out)))
#         print(out.shape)
        out = out.view(-1, 8*8*32)
        out = self.act(self.fc1(out))
        out = self.act(self.fc2(out))
        
        return self.fc3(out)


In [ ]:
class TrainingModuleBase():
    """
    Training Module to train the model
    """
    
    def __init__(self, model):
        self.model = model
        self.loss_fn = nn.BCEWithLogitsLoss()
        self.optimizer = optim.Adam(self.model.parameters())
        self.cuda = False
        if torch.cuda.is_available():
            self.cuda = True
            self.model = model.cuda()
    
    def train_epoch(self, epoch, train_iterator):
        total_loss = 0
        stats_string = "Epoch : {:3d} | Iteration : {:4d} | Loss : {:4.4f}"
        for i, data in enumerate(train_iterator):
            inputs = data['image']
            labels = data['label'].float()
            
            if self.cuda:
                inputs = inputs.cuda()
                labels = labels.cuda()
            
            
            self.optimizer.zero_grad()
            
            outputs = self.model(inputs).squeeze(1)
            loss = self.loss_fn(outputs, labels)
            loss.backward()
            self.optimizer.step()
            
            total_loss += loss.item()
            
            if i % 50 == 0:
                print(stats_string.format(epoch+1, i+1, total_loss/50))
                total_loss = 0
    
    def train_model(self, train_iterator, dev_iterator, num_epocs = 3):
        self.model.train()
        max_accuracy = float('-inf')
        stats_string = "Epoch : {:2d} | Train Accuracy : {:4.4f} | Dev Accuracy : {:4.4f}"
        for i in range(num_epocs):
            self.train_epoch(i, train_iterator)
            dev_acc = self.evaluate(dev_iterator)
            train_acc = self.evaluate(train_iterator)
            print(stats_string.format(i, train_acc, dev_acc))
            if dev_acc > max_accuracy:
                best_model = copy.deepcopy(self.model)
                max_accuracy = dev_acc
        
        print("Finished Training")
        
        return best_model
    
    def evaluate(self, iterator):
        self.model.eval()
        
        correct = 0
        total = 0
        
        for i, data in enumerate(iterator):
            inputs = data['image']
            labels = data['label'].float()
            
            if self.cuda:
                inputs = inputs.cuda()
                labels = labels.cuda()
            
            preds = self.model(inputs).squeeze(1)
            
            correct = (torch.round(torch.sigmoid(preds)) == labels).sum()
            total = inputs.shape[0]
        
        return correct.item()/total
            

In [ ]:
base_model = CactusIdentifierBase()
print(base_model)

In [ ]:
trainer = TrainingModuleBase(base_model)

In [ ]:
trainer.evaluate(dev_loader)

In [ ]:
best_model_base = trainer.train_model(train_loader, dev_loader)

In [ ]:
test_set = CactusImageDataset("data/sample_submission.csv", "data/test/", image_transforms['test'])
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
final_preds = []
for i, data in enumerate(test_loader):
    inputs = data['image']
    labels = data['label'].float()

    if torch.cuda.is_available():
        inputs = inputs.cuda()
        labels = labels.cuda()

    preds = self.model(inputs).squeeze(1)
    pred = torch.sigmoid(preds)
    final_preds += pred.tolist()

test_dataset.cactus_annotations['has_cactus'] = final_preds
# test_dataset.cactus_annotations.to_csv('samle_submission.csv', index=False)


## New Model and Training Module

In [ ]:
class CactusIdentifier(nn.Module):
    """
        Aerial Cactus Identifier Model
    """
    def __init__(self):
        super(CactusIdentifier, self).__init__()
        self.conv1 = nn.Conv2d(3,16,3, padding = 1)
        self.conv2 = nn.Conv2d(16,32,3, padding = 1)
        self.conv3 = nn.Conv2d(32,64,3, padding = 1)
        self.conv4 = nn.Conv2d(64,128,3, padding = 1)
        self.pool = nn.MaxPool2d(2,2)
        
        self.fc1 = nn.Linear(128*2*2, 512)
        self.fc2 = nn.Linear(512, 2)
        self.act = nn.ReLU()
        self.drop = nn.Dropout(0.25)
    
    def forward(self, inp_image):
        out = self.pool(self.act(self.conv1(inp_image)))
        out = self.pool(self.act(self.conv2(out)))
        out = self.pool(self.act(self.conv3(out)))
        out = self.pool(self.act(self.conv4(out)))
        
        out = out.view(-1, 128*2*2)
        out = self.drop(out)
        out = self.act(self.fc1(out))
        out = self.drop(out)
        out = self.fc2(out)
        
        return out


In [ ]:
class TrainingModule():
    """
    Training Module to train the model
    """
    
    def __init__(self, model):
        self.model = model
        self.loss_fn = nn.CrossEntropyLoss()
        self.optimizer = optim.Adam(self.model.parameters())
        self.is_cuda = False
        if torch.cuda.is_available():
            self.is_cuda = True
            self.model = model.cuda()
    
    def train_epoch(self, epoch, train_iterator):
        total_loss = 0
        stats_string = "Epoch : {:3d} | Iteration : {:4d} | Loss : {:4.4f}"
        for i, data in enumerate(train_iterator):
            inputs = data['image']
            labels = data['label'].long()
            
            if self.is_cuda:
                inputs = inputs.cuda()
                labels = labels.cuda()
                    
            self.optimizer.zero_grad()
        
            outputs = self.model(inputs)
            loss = self.loss_fn(outputs, labels)
            loss.backward()
            self.optimizer.step()
            
            total_loss += loss.item()
            
            if i % 100 == 0:
                print(stats_string.format(epoch+1, i, total_loss/50))
                total_loss = 0
    
    def train_model(self, train_iterator, dev_iterator, num_epocs = 25):
        self.model.train()
        min_loss = 1000000
        stats_string = "Epoch : {:2d} | Train Loss : {:4.4f} | Dev Loss : {:4.4f}"
        for i in range(num_epocs):
            self.train_epoch(i, train_iterator)
            dev_loss = self.evaluate(dev_iterator)
            train_loss = self.evaluate(train_iterator)
            print(stats_string.format(i+1, train_loss, dev_loss))
            if dev_loss <= min_loss:
                best_model = copy.deepcopy(self.model)
                min_loss = dev_loss
    
        print("Finished Training")
        
        return best_model
    
    def evaluate(self, iterator):
        self.model.eval()
        
        loss_total = 0
        
        for i, data in enumerate(iterator):
            inputs = data['image']
            labels = data['label'].long()
            
            if self.is_cuda:
                inputs = inputs.cuda()
                labels = labels.cuda()
            
            preds = self.model(inputs)
            loss = self.loss_fn(preds, labels)
            loss_total += loss.item()
        
        return loss_total/(i+1)
            

In [ ]:
base_model = CactusIdentifier()
print(base_model)

In [ ]:
trainer = TrainingModule(base_model)

In [ ]:
best_model = trainer.train_model(train_loader, dev_loader)

In [ ]:
test_set = CactusImageDataset("data/sample_submission.csv", "data/test/", image_transforms['test'])
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
final_preds = []
for i, data in enumerate(test_loader):
    inputs = data['image']
    labels = data['label'].long()

    if torch.cuda.is_available():
        inputs = inputs.cuda()
        labels = labels.cuda()

    preds = best_model(inputs)
    pred = preds[:,1]
    final_preds += pred.tolist()
    break
    
test_set.cactus_annotations['has_cactus'] = final_preds
test_set.cactus_annotations.head()